In [ ]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
df = pd.read_csv(r"C:\Users\anish\Data Science with Gen AI\Task Codes\XGB Hyperparameter tuning\Wholesale customers data (1).csv")

In [ ]:
X = df.drop('Channel', axis=1)

y = df['Channel']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
y[y == 2] = 0

y[y == 1] = 1

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [ ]:
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics import accuracy_score

def objective(space):
    clf = XGBClassifier(
        n_estimators=int(space['n_estimators']),
        max_depth=int(space['max_depth']),
        gamma=space['gamma'],
        reg_alpha=space['reg_alpha'],
        min_child_weight=space['min_child_weight'],
        colsample_bytree=space['colsample_bytree'],
        eval_metric="auc",
        use_label_encoder=False
    )

    # Convert to numpy arrays
    X_train_np = np.array(X_train)
    y_train_np = np.array(y_train)
    X_test_np = np.array(X_test)
    y_test_np = np.array(y_test)

    eval_set = [(X_train_np, y_train_np), (X_test_np, y_test_np)]

    # Perform fitting without passing early_stopping_rounds to fmin
    clf.fit(
        X_train_np,
        y_train_np,
        eval_set=eval_set,
        early_stopping_rounds=10,  # No longer an issue here
        verbose=False
    )

    # Predictions and evaluation
    preds = clf.predict(X_test_np)
    accuracy = accuracy_score(y_test_np, preds > 0.5)
    print("SCORE:", accuracy)

    return {'loss': -accuracy, 'status': STATUS_OK}


In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,space = space,algo = tpe.suggest,max_evals = 100,trials = trials)